<h1>Test set path</h1>

In [6]:
import math
test_set = "/home/theo/Documents/Unif/Master/Chimprec - Extra/Detection - test set" #insert path

<h3>Generic function to compute intersection over union.</h3>

In [3]:
def yolo_to_relative_coord(bbox, img_dim):
    x_center, y_center, width, height = bbox
    img_w, img_h = img_dim
    
    x_min = (x_center - width / 2) * img_w
    y_min = (y_center - height / 2) * img_h
    x_max = (x_center + width / 2) * img_w
    y_max = (y_center + height / 2) * img_h
    
    return [x_min, y_min, x_max, y_max]

In [19]:
"""
input:
bbox1, bbox2: bounding boxes in YOLO format
YOLO format ==> [x_center, y_center, width, height], normailzed coordinates

bbox1 is expected to be the ground truth
bbox2 is expected to be the prediction

output:
The intersection over union metric between bbox1 and bbox2
"""
def iou(bbox1, bbox2, img_dim=(1080, 1920)):

    # conversion into relative coordinates
    Ax, Ay, Bx, By = yolo_to_relative_coord(bbox1, img_dim)
    Cx, Cy, Dx, Dy = yolo_to_relative_coord(bbox2, img_dim)

    # computation of the intersection
    x_overlap = min(Dx, Bx) - max(Ax, Cx)
    y_overlap = min(Dy, By) - max(Ay, Cy)
    if (x_overlap < 0 or y_overlap < 0): return 0 # no overlap case
    intersection = x_overlap*y_overlap

    # computation of the union
    area_1 = abs((Bx-Ax)*(By-Ay))
    area_2 = abs((Dx-Cx)*(Dy-Cy))
    union = area_1 + area_2 - intersection

    # IoU
    return intersection/union

def weighted_iou(bbox1, bbox2, img_dim=(1080, 1920)):
    iou_score = iou(bbox1, bbox2, img_dim)
    w = weight(area_covered(bbox2, img_dim))
    weighted_score = iou_score*w
    if (weighted_score <= 0): return 0
    elif (weighted_score >= 1): return 1
    else: return weighted_score

def area_covered(bbox, img_dim):
    
    img_height, img_width = img_dim
    _, _, bbox_width, bbox_height = bbox

    bbox_area = (bbox_width * img_width) * (bbox_height * img_height)
    image_area = img_width * img_height

    return bbox_area / image_area

def weight(bbox_size):

    def h(x):
        return 0.4 * math.tanh(15*x + 0.125) + 0.5
    
    return 1/h(bbox_size) - 0.112


In [ ]:
print(iou([0.386891, 0.573836, 0.274225, 0.328799], [0.324225, 0.497304, 0.400000, 0.482353])) # BIG
print(iou([0.670643, 0.656066, 0.136706, 0.164583], [0.639254, 0.618382, 0.199631, 0.240196])) # MEDIUM
print(iou([0.820532, 0.697243, 0.068833, 0.082230], [0.805022, 0.678125, 0.100000, 0.120956])) # SMALL
print(iou([0.902880, 0.717892, 0.033973, 0.040931], [0.895362, 0.709050, 0.049453, 0.058860])) # TINY

0.46731805220968897
0.46922246572820714
0.4679501297992655
0.4777199233072285


In [20]:
print(weighted_iou([0.386891, 0.573836, 0.274225, 0.328799], [0.324225, 0.497304, 0.400000, 0.482353])) # BIG
print(weighted_iou([0.670643, 0.656066, 0.136706, 0.164583], [0.639254, 0.618382, 0.199631, 0.240196])) # MEDIUM
print(weighted_iou([0.820532, 0.697243, 0.068833, 0.082230], [0.805022, 0.678125, 0.100000, 0.120956])) # SMALL
print(weighted_iou([0.902880, 0.717892, 0.033973, 0.040931], [0.895362, 0.709050, 0.049453, 0.058860])) # TINY

0.4680035126804085
0.5527210857069486
0.7037188888519735
0.7892841904481294


In [32]:
area_covered([0.267645, 0.307458, 0.523305, 0.590944], (1080, 1920))

641248.254554112
2073600


0.30924394992

<h3><span style="color:green;">Test of the <i>iou</i> function.</span></h3>

In [4]:
def test(observed, expected, test_name=""):
    msg = f'({test_name}): Observed: {observed}; Expected: {expected}'
    if (round(observed, 5) == round(expected, 5)): print(f"Test passed ({test_name})")
    else: print(f"Test failed{msg}")

In [5]:
bbox = [0.1, 0.1, 0.2, 0.2]
test(iou(bbox, bbox), 1, "Perfect Overlap") # perfect overlap

bbox2 = [0.9, 0.9, 0.1, 0.1] 
test(iou(bbox, bbox2), 0, "No Overlap") # no overlap

bbox3 = [0.1, 0.2, 0.2, 0.2] 
test(iou(bbox, bbox3), 1/3, "No Overlap") # no overlap

Test passed (Perfect Overlap)
Test passed (No Overlap)
Test passed (No Overlap)


<h2>Computation of the following metrics based on the ground truth (from the test set) and the predictions</h2>
<h3>
    <ul>
        <li>True Positive</li>
        <li>False Positive</li>
        <li><span style="color:grey;">True Negative: not counted (see below)</span></li>
        <li>False Negative</li>
    </ul> 
</h3>

In [ ]:
"""
The arguments <ground_truths> and <predictions> are expected to be 
python dictionaries structured as follows:

    dictionnary {
        image_name 1: [
                bbox 1,
                bbox 2, 
                ...
            ],
        image_name 2: [
                bbox 1,
                bbox 2, 
                ...
            ],
        ...
    }

PS: the bounding boxes are expected to be encoded in YOLO format.

    
For each image, every prediction bbox will be 
compared to every ground truth bbox to find the best match.

If a ground truth bbox finds no match among the prediction bbox:
+1 false negative

If a prediction bbox finds no match among the ground truth bbox:
+1 false positive

If the best match between a prediction bbox and the ground truth
bboxes has a low IoU (bellow a given threshold <t>):
+1 false positive

If a prediction bbox best match in the ground truth bboxes has
already been matched by another prediction bbox with a greater score:
+1 false positive

If the best match between a prediction bbox and the ground truth
bboxes has a high IoU (above a given threshold <t>):
+1 true positive

The true negative won't be counted because it means that no
prediction bbox has been found in the background. This is nonsense to count this.
"""
def extract_metrics(ground_truths:dict, predictions:dict, t=0.75):
    tp, fp, fn = 0, 0, 0  # Initialize counters for TP, FP, and FN
    
    # Iterate over all unique image names in ground truths and predictions
    for image_name in set(ground_truths.keys()).union(predictions.keys()):
        gt_bboxes = ground_truths.get(image_name, [])  # Retrieve ground truth bboxes (default empty list if missing)
        pred_bboxes = predictions.get(image_name, [])  # Retrieve predicted bboxes (default empty list if missing)
        
        matched_gt = set()  # Store indices of matched ground truth bboxes
        pred_matched_scores = []  # Keep track of IoU scores of matched predictions
        
        # Iterate over each predicted bounding box
        for pred in pred_bboxes:
            best_iou = 0  # Initialize the best IoU score for the current prediction
            best_gt_idx = -1  # Index of the best-matching ground truth bbox
            
            # Compare prediction with each ground truth bbox
            for i, gt in enumerate(gt_bboxes):
                score = weighted_iou(pred, gt)  # Compute IoU including the weighing
                if score > best_iou:  # Update best match if IoU is higher
                    best_iou = score
                    best_gt_idx = i
            
            # Determine if the prediction is a TP or FP based on IoU and previous matches
            if best_iou >= t and best_gt_idx not in matched_gt:
                matched_gt.add(best_gt_idx)  # Mark ground truth bbox as matched
                pred_matched_scores.append(best_iou)  # Store IoU score
                tp += 1
            else:
                fp += 1
        
        # Count False Negatives (ground truths that were not matched)
        fn += len(gt_bboxes) - len(matched_gt)
    
    return {"true_positives": tp, "false_positives": fp, "false_negatives": fn}

<h3><span style="color:green;">Test of the <i>extract_metrics</i> function.</span></h3>

In [ ]:
print("\n------------------------------------")
# perfect match
GT = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
PRED = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "perfect match (TP)")
test(fp, 0, "perfect match (FP)")
test(fn, 0, "perfect match (FN)")

print("\n------------------------------------")
# No overlap
GT = {"img2": [[0.8, 0.8, 0.1, 0.1]]}
PRED = {"img2": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap (TP)")
test(fp, 1, "No overlap (FP)")
test(fn, 1, "No overlap (FN)")

print("\n------------------------------------")
# No overlap but bboxes are closer
GT = {"img3": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img3": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap but bboxes are closer (TP)")
test(fp, 1, "No overlap but bboxes are closer (FP)")
test(fn, 1, "No overlap but bboxes are closer (FN)")

print("\n------------------------------------")
# Not a perfect match but above threshold
GT = {"img4": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img4": [[0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Not a perfect match but above threshold (TP)")
test(fp, 0, "Not a perfect match but above threshold (FP)")
test(fn, 0, "Not a perfect match but above threshold (FN)")

print("\n------------------------------------")
# Two prediction bboxes matching the same ground truth
GT = {"img5": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# Composed case - 2 pred matching 1 GT - 1 pred matching 1 GT
GT = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]]}
PRED = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 2, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# One prediction overlapping two ground truths
GT = {"img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]}
PRED = {"img7": [[0.367304, 0.808140, 0.362103, 0.366419]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 0, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 1, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# GT completed included within prediction
GT = {"img8": [[0.5, 0.5, 0.5, 0.5]]}
PRED = {"img8": [[0.5, 0.5, 0.55, 0.55]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "GT completed included within prediction (TP)")
test(fp, 0, "GT completed included within prediction (FP)")
test(fn, 0, "GT completed included within prediction (FN)")


------------------------------------
Test passed (perfect match (TP))
Test passed (perfect match (FP))
Test passed (perfect match (FN))

------------------------------------
Test passed (No overlap (TP))
Test passed (No overlap (FP))
Test passed (No overlap (FN))

------------------------------------
Test passed (No overlap but bboxes are closer (TP))
Test passed (No overlap but bboxes are closer (FP))
Test passed (No overlap but bboxes are closer (FN))

------------------------------------
Test passed (Not a perfect match but above threshold (TP))
Test passed (Not a perfect match but above threshold (FP))
Test passed (Not a perfect match but above threshold (FN))

------------------------------------
Test passed (Two prediction bboxes matching the same ground truth (TP))
Test passed (Two prediction bboxes matching the same ground truth (FP))
Test passed (Two prediction bboxes matching the same ground truth (FN))

------------------------------------
Test passed (Two prediction bboxes

In [71]:
print(iou([0.314258, 0.373674, 0.406016, 0.410403], [0.267645, 0.307458, 0.523305, 0.590944])) # biggest bboxes
print(iou([0.586441, 0.666424, 0.064664, 0.086875], [0.577793, 0.655729, 0.090977, 0.124319]))
print(iou([0.803359, 0.314882, 0.008266, 0.014708], [0.801855, 0.312201, 0.017289, 0.030736])) # smallest bboxes

0.5388308631134302
0.4966932563688805
0.22878724060449573


In [72]:
print(weighted_iou([0.314258, 0.373674, 0.406016, 0.410403], [0.267645, 0.307458, 0.523305, 0.590944])) # biggest bboxes
print(weighted_iou([0.586441, 0.666424, 0.064664, 0.086875], [0.577793, 0.655729, 0.090977, 0.124319]))
print(weighted_iou([0.803359, 0.314882, 0.008266, 0.014708], [0.801855, 0.312201, 0.017289, 0.030736])) # smallest bboxes

0.5383519023462183
0.7604312260034591
0.611554714781719


<h3><span style="color:green;">Combined test of the <i>extract_metrics</i> function.</span></h3>

In [73]:
GT = {
    "img1": [[0.2, 0.2, 0.1, 0.1]], 
    "img2": [[0.8, 0.8, 0.1, 0.1]], 
    "img3": [[0.1, 0.1, 0.1, 0.1]],
    "img4": [[0.1, 0.1, 0.1, 0.1]],
    "img5": [[0.1, 0.1, 0.1, 0.1]],
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]],
    "img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]
    }
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }

tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Combined test (TP)")
test(fp, 4, "Combined test (FP)")
test(fn, 3, "Combined test (FN)")

Test passed (Combined test (TP))
Test passed (Combined test (FP))
Test passed (Combined test (FN))


<h2>Extract the data from the test set (to <i>dict</i>)</h2>

In [47]:
import os

def extract_ground_truth(test_set_path = test_set):
    path = f'{test_set_path}/labels/obj_train_data'
    data = dict()

    # check whether the specified path is valid directory
    if not os.path.isdir(path):
        print(f"Error: {path} is not a valid directory.")
        return
    
    for textfile in os.listdir(path):
        file_path = f"{path}/{textfile}"

        # check whether textfile exists in the folder located at the specified path
        if os.path.isfile(file_path):
            data[textfile.strip(".txt")] = []

            # open the file in read mode
            with open(file_path, 'r') as file:

                # extract the bounding boxes one by one
                for line in file.readlines():
                    splitted = line.split(" ")
                    detection_class = splitted[0]
                    if (detection_class == 0): continue # abort iteration if the class is a face
                    bbox = splitted[1:]
                    for i in range(len(bbox)): bbox[i] = float(bbox[i].strip("\n"))
                    data[textfile.strip(".txt")].append(bbox)
                file.close()
    
    return data


<h3><span style="color:green;">Test of the <i>extract_ground_truth</i> function.</span></h3>

In [ ]:
GT = extract_ground_truth(test_set_path="test_set_example")
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Extract ground truth from files (TP)")
test(fp, 4, "Extract ground truth from files (FP)")
test(fn, 3, "Extract ground truth from files (FN)")

Test passed (Extract ground truth from files (TP))
Test passed (Extract ground truth from files (FP))
Test passed (Extract ground truth from files (FN))
{'img1': [[0.2, 0.2, 0.1, 0.1]], 'img2': [[0.8, 0.8, 0.1, 0.1]], 'img3': [[0.1, 0.1, 0.1, 0.1]], 'img4': [[0.1, 0.1, 0.1, 0.1]], 'img5': [[0.1, 0.1, 0.1, 0.1]], 'img6': [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]], 'img7': [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.34654]]}


In [13]:
def convert_to_yolo(bbox, img_dim=(1080, 1920)):
    """
    Convert bounding box from (x1, y1, x2, y2) format to YOLO format.

    Args:
        bbox (tuple): Bounding box in (x1, y1, x2, y2) format.
        img_dim (tuple): Image dimensions in (width, height) format.

    Returns:
        tuple: (x_center, y_center, width, height) in YOLO format, normalized to [0,1].
    """
    x1, y1, x2, y2 = bbox
    img_width, img_height = img_dim

    # Compute center coordinates
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0

    # Compute width and height
    width = x2 - x1
    height = y2 - y1

    # Normalize values by image dimensions
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return [round(x_center, 4), round(y_center, 4), round(width, 4), round(height, 4)]

<h1>Effective code</h1>

<h3>Ground truth extraction.</h3>

In [16]:
GT = extract_ground_truth()

<h3>Predictions extraction</h3>

<h2>Visualisation functions.</h2>

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

model_path = "/home/theo/Documents/Unif/Master/ChimpRec/Code/Body_detection/YOLO_small/runs/detect/train9/weights/best.pt"

def predict(model_path, test_set_path):
    """
    Predict bounding boxes using a YOLO model.

    Args:
        model_path (str): Path to the YOLO model.
        test_set_path (str): Path to the test set directory.

    Returns:
        dict: Dictionary with image prefixes as keys and predicted bounding boxes (YOLO format).
    """
    predictions = dict()

    # Load the YOLO model
    model = YOLO(model_path)

    test_set_path = f"{test_set_path}/images"

    # Ensure the test set path exists
    if not os.path.isdir(test_set_path):
        print(f"Error: Test set directory '{test_set_path}' not found.")
        return {}

    # Iterate over all images in the test set directory
    image_extension = ".png"
    for filename in sorted(os.listdir(test_set_path)):
        file_path = os.path.join(test_set_path, filename)

        # Check if the file is an image
        if not filename.lower().endswith(image_extension):
            continue

        filename_prefix = filename.split(".")[0]
        predictions[filename_prefix] = []

        # Run inference
        results = model(file_path)[0]

        # Extract YOLO format bounding boxes
        img = cv2.imread(file_path)
        img_height, img_width, _ = img.shape

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            # Convert bbox to YOLO format (normalized)
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            predictions[filename_prefix].append((x_center, y_center, width, height))

    return predictions

predictions = predict(model_path, test_set)


image 1/1 /home/theo/Documents/Unif/Master/Chimprec - Extra/Detection - test set/images/20241015 - 11h46_frame_11231.png: 384x640 3 Bodys, 58.7ms
Speed: 1.8ms preprocess, 58.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/theo/Documents/Unif/Master/Chimprec - Extra/Detection - test set/images/20241015 - 11h46_frame_12495.png: 384x640 3 Bodys, 86.9ms
Speed: 1.4ms preprocess, 86.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/theo/Documents/Unif/Master/Chimprec - Extra/Detection - test set/images/20241015 - 11h46_frame_13051.png: 384x640 5 Bodys, 78.6ms
Speed: 1.9ms preprocess, 78.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/theo/Documents/Unif/Master/Chimprec - Extra/Detection - test set/images/20241015 - 11h46_frame_13182.png: 384x640 3 Bodys, 81.6ms
Speed: 1.8ms preprocess, 81.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/theo/Do

<h3>Visualisation support</h3>

In [ ]:
def draw_predictions(predictions, ground_truth, test_set_path):
    """
    Draws predicted and ground truth bounding boxes on images and displays them in Jupyter Notebook.

    Args:
        predictions (dict): Dictionary of predicted bounding boxes in YOLO format.
        ground_truth (dict): Dictionary of ground truth bounding boxes in YOLO format.
        test_set_path (str): Path to the test set directory.

    Returns:
        None
    """
    test_set_path = f"{test_set_path}/images"

    for img_prefix, pred_bboxes in predictions.items():
        img_name = f"{img_prefix}.png"
        img_path = os.path.join(test_set_path, img_name)

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error: Unable to load image {img_name}")
            continue

        # Convert from BGR to RGB for correct color display
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width, _ = image.shape

        def draw_bbox(image, bbox, color, label=None):
            """ Draws a single bounding box on the image. """
            print(bbox)
            x_center, y_center, width, height = bbox

            # Convert YOLO format to absolute pixel coordinates
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)

            # Draw rectangle
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

            # Put label
            if label:
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness = 1
                cv2.putText(image, label, (x1, max(y1 - 5, 10)), font, font_scale, color, thickness, cv2.LINE_AA)

        # Draw ground truth boxes (green)
        if img_prefix in ground_truth:
            for bbox in ground_truth[img_prefix]:
                print(bbox)
                draw_bbox(image, bbox, (0, 255, 0))  # Class label only

        # Draw predicted boxes (blue)
        for bbox in pred_bboxes:
            draw_bbox(image, bbox, (255, 0, 0), f"P {int(bbox[0])}")  # Class & confidence

        # Display image
        plt.figure(figsize=(10, 6))
        plt.imshow(image)
        plt.axis("off")
        plt.title(f"Predictions & Ground Truth for {img_name}")
        plt.show()

# Example usage:
# predictions = predict(model_path, test_set)
draw_predictions(predictions, GT, test_set)



<h3>Computation of the model performance</h3>

In [46]:
print(extract_metrics(GT, predictions, t=0.6))

tp, fp, fn = extract_metrics(GT, predictions, t=0.6).values()
print(f"precision={tp/(tp+fp)}")
print(f"recall={tp/(tp+fn)}")

{'true_positives': 718, 'false_positives': 262, 'false_negatives': 475}
precision=0.7326530612244898
recall=0.6018440905280805
